# Documents and Document Loaders

In [1]:
from langchain_core.documents import Document

In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
from langchain_community.document_loaders import UnstructuredXMLLoader

In [4]:
file_path = "C:/Users/Aaditya_More/Downloads/coding_dell/rag-main/cleaned_xml.xml"

In [5]:
loader = UnstructuredXMLLoader(file_path)

In [6]:
docs = loader.load()

In [7]:
docs[0].metadata

{'source': 'C:/Users/Aaditya_More/Downloads/coding_dell/rag-main/cleaned_xml.xml'}

# Splitting

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [10]:
all_splits = text_splitter.split_documents(docs)

In [11]:
all_splits[0]

Document(metadata={'source': 'C:/Users/Aaditya_More/Downloads/coding_dell/rag-main/cleaned_xml.xml', 'start_index': 0}, page_content='2.5.46\n\n89.3.1\n\nLatitude 9420\n\nSetup\n\nUEFIBO\n\n0\n\n5\n\nWindows Boot Manager\n\n0\n\nInternal UEFI Shell\n\n1\n\nUEFI PM991a NVMe Samsung 128GB S65YNE0R113210 1\n\n6\n\nUEFI PM991a NVMe Samsung 128GB S65YNE0R113210 1 2\n\n7\n\nDUAFBOOT\n\n2\n\nUEFI HTTPs Boot\n\nAdminPassword\n\n[RegEx:^\\\\133\\\\x20-\\\\x7e\\\\135{0,32}$]\n\n0\n\nAdmin Password\n\nSystemPassword\n\n[RegEx:^\\\\133\\\\x20-\\\\x7e\\\\135{0,32}$]\n\n0\n\nSystem Password\n\nPxeBootPriority\n\n0\n\nEnable PXE Boot Priority\n\nWhen this option is enabled: If a new PXE boot option is detected, it will be added to the top of the Boot Sequence.\n\nDisabled\n\nDisabled\n\nEnabled\n\nDisabled\n\nAdvancedMode\n\n0\n\nBIOS Setup Advanced Mode\n\nDisabling BIOS Setup Advanced Mode will allow access to basic BIOS settings only.Enabling BIOS Setup Advanced Mode will make both basic BIOS sett

# Embeddings

In [22]:
from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(model="llama3.1", num_thread=12)
embeddings = OllamaEmbeddings(model="mxbai-embed-large", num_thread=12)

In [23]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

In [24]:
assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1024

[-0.026513359, 0.04238228, 0.023390831, 0.00047690442, -0.014362901, 0.017256204, 0.014094396, -0.014118202, 0.00017724487, 0.07668811]


# Vector stores

In [25]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [26]:
ids = vector_store.add_documents(documents=all_splits)

# Usage

In [29]:
results = vector_store.similarity_search(
    "what are the camera options"
)

print(results[0])

page_content='Disabled

Disabled

Enabled

Disabled

SecureBootMode

0

Secure Boot Mode

Changes to the Secure Boot operation mode modifies the behavior of Secure Boot to allow evaluation or enforcement of UEFI driver signatures. Deployed Mode should be selected for normal operation of Secure Boot.Deployed Mode - Checks the integrity of UEFI drivers and bootloaders before allowing execution. Use this mode for full Secure Boot protections.Audit Mode - Performs a signature check but does not block execution of all UEFI drivers and bootloaders.  Use this mode when making modifications to Secure Boot Keys.

DeployedMode

DeployedMode

AuditMode

DeployedMode

Camera

0

Enable Camera

Click the checkbox to enable the Camera.

Enabled

Disabled

Enabled

Enabled

IntegratedAudio

0

Enable Audio

Use this setting to switch all integrated audio On/Off, or enable/disable the microphone and the internal speaker separately.

Enabled

Disabled

Enabled

Enabled

Microphone

0

Enable Microphone

In [31]:
len(results)

4

In [32]:
results = vector_store.similarity_search_with_score("How do i disable camera")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6185881459774383

page_content='Disabled

Disabled

Enabled

Disabled

SecureBootMode

0

Secure Boot Mode

Changes to the Secure Boot operation mode modifies the behavior of Secure Boot to allow evaluation or enforcement of UEFI driver signatures. Deployed Mode should be selected for normal operation of Secure Boot.Deployed Mode - Checks the integrity of UEFI drivers and bootloaders before allowing execution. Use this mode for full Secure Boot protections.Audit Mode - Performs a signature check but does not block execution of all UEFI drivers and bootloaders.  Use this mode when making modifications to Secure Boot Keys.

DeployedMode

DeployedMode

AuditMode

DeployedMode

Camera

0

Enable Camera

Click the checkbox to enable the Camera.

Enabled

Disabled

Enabled

Enabled

IntegratedAudio

0

Enable Audio

Use this setting to switch all integrated audio On/Off, or enable/disable the microphone and the internal speaker separately.

Enabled

Disabled

Enabled

Enabled

Micro

In [33]:
embedding = embeddings.embed_query("How do i disable camera")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Disabled

Disabled

Enabled

Disabled

SecureBootMode

0

Secure Boot Mode

Changes to the Secure Boot operation mode modifies the behavior of Secure Boot to allow evaluation or enforcement of UEFI driver signatures. Deployed Mode should be selected for normal operation of Secure Boot.Deployed Mode - Checks the integrity of UEFI drivers and bootloaders before allowing execution. Use this mode for full Secure Boot protections.Audit Mode - Performs a signature check but does not block execution of all UEFI drivers and bootloaders.  Use this mode when making modifications to Secure Boot Keys.

DeployedMode

DeployedMode

AuditMode

DeployedMode

Camera

0

Enable Camera

Click the checkbox to enable the Camera.

Enabled

Disabled

Enabled

Enabled

IntegratedAudio

0

Enable Audio

Use this setting to switch all integrated audio On/Off, or enable/disable the microphone and the internal speaker separately.

Enabled

Disabled

Enabled

Enabled

Microphone

0

Enable Microphone

In [35]:
## async query
results = await vector_store.asimilarity_search("How do i disable camera")

print(results[0])

page_content='Disabled

Disabled

Enabled

Disabled

SecureBootMode

0

Secure Boot Mode

Changes to the Secure Boot operation mode modifies the behavior of Secure Boot to allow evaluation or enforcement of UEFI driver signatures. Deployed Mode should be selected for normal operation of Secure Boot.Deployed Mode - Checks the integrity of UEFI drivers and bootloaders before allowing execution. Use this mode for full Secure Boot protections.Audit Mode - Performs a signature check but does not block execution of all UEFI drivers and bootloaders.  Use this mode when making modifications to Secure Boot Keys.

DeployedMode

DeployedMode

AuditMode

DeployedMode

Camera

0

Enable Camera

Click the checkbox to enable the Camera.

Enabled

Disabled

Enabled

Enabled

IntegratedAudio

0

Enable Audio

Use this setting to switch all integrated audio On/Off, or enable/disable the microphone and the internal speaker separately.

Enabled

Disabled

Enabled

Enabled

Microphone

0

Enable Microphone

# Retrievers